In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import requests
import tweepy
import io
import os

In [10]:
import json
from timeit import default_timer as timer

## Gather

In [9]:
#Create 'raw-data' folder
folder_name = 'raw-data'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [12]:
archive = pd.read_csv('raw-data/twitter-archive-enhanced.csv')

In [7]:
#Download image_predictions.tsv from the url
url =  "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
response = requests.get(url)

In [96]:
img_predit = pd.read_csv(io.StringIO(response.content.decode('utf-8')), sep='\t')

In [16]:
consumer_key = 'JcrI6rjL8V9htFTOrSvdn1SJk'
consumer_secret = '7GCzjEvuHghj2mkf7e6SnwLavz3UJCjtRVCLWjTwM7M1wxxjF3'
access_token = '707089529543401472-NNY42Am0BwX2hL7iQfZhrLP3FzKOIWJ' 
access_secret = 'aUpj7Np57ZszDVJZpK4IDMmszMMTAIh4Jdgvs9wHryhU4'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

In [69]:
start = timer()
i=0
for tweet_id in tweeter_archive['tweet_id']:
    if i<100000:
        with open (os.path.join(folder_name,
                           'tweet_json.txt'), mode='a') as file:
            try:
                json.dump(api.get_status(tweet_id,
                                         tweet_mode='extended',
                                         wait_on_rate_limit=True,
                                         wait_on_rate_limit_notify=True)._json, file)
                file.write('\n')
                #print('Tweet record with ID {} was written to file'.format(tweet_id))
            except:
                print('No status found with ID {}.'.format(tweet_id))
    i = i+1
end = timer()
print('Total runing time of the for loop is {}s'.format(end - start))

No status found with ID 888202515573088257.
No status found with ID 873697596434513921.
No status found with ID 872668790621863937.
No status found with ID 872261713294495745.
No status found with ID 869988702071779329.
No status found with ID 866816280283807744.
No status found with ID 861769973181624320.
No status found with ID 845459076796616705.
No status found with ID 842892208864923648.
No status found with ID 837012587749474308.
No status found with ID 827228250799742977.
No status found with ID 812747805718642688.
No status found with ID 802247111496568832.


Rate limit reached. Sleeping for: 479


No status found with ID 775096608509886464.
No status found with ID 770743923962707968.
No status found with ID 754011816964026368.


Rate limit reached. Sleeping for: 644


No status found with ID 680055455951884288.
Total runing time of the for loop is 2030.1335412909975s


In [351]:
with open ('raw-data/tweet_json.txt') as file:
    lines = file.readlines()
    tweet_json = pd.DataFrame()
    for line in lines:
        tweet_json = tweet_json.append(json.loads(line), ignore_index=True)

# Access and Clean

## Tweet-Archive Table

In [295]:
#Set jupyter notebook to display full text
pd.set_option('display.max_colwidth', -1)

In [384]:
archive_clean = archive.copy()

In [385]:
archive_clean.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

#### Looks like all the column names are in good format

### Quality Issue #1 Drop the re-tweet rows

#### Define
Select only rows where `retweeted_status_id` is `NaN` 

#### Code

In [386]:
archive_clean = archive_clean[archive_clean['retweeted_status_id'].isnull()]

#### Test
Make sure the `retweeted_status_user_id` and `retweeted_status_timestamp` are all `NaN` as well

In [387]:
#Make sure all the 'retweeted_status_user_id' is NaN
archive_clean['retweeted_status_user_id'].isnull().all()

True

In [388]:
#Make sure all the 'retweeted_status_timestamp' is NaN
archive_clean['retweeted_status_timestamp'].isnull().all()

True

### Quality Issue #2 Drop rows without pictures

For the breed prediction neural network to work properly, a image is required for each tweet. As a result I am going to drop rows without pictures

#### Define
Keep rows which contains `https://twitter.com/dog_rates/status/../photo/1` in the `expanded_url` column, there might be other strings before or after the photo url, replace NaN with False value.

#### Code

In [389]:
has_img = archive_clean['expanded_urls'].str.match(r'.*,?https://twitter.com/dog_rates/status/\d{18}/photo/1,?.*', case=False, na=False)

In [390]:
archive_clean[~has_img].to_csv('no_img.csv')

In [391]:
archive_clean = archive_clean[has_img]

#### Test
Physically check the `no_img.csv` file with excel and make sure I didn't drop any rows with image

In [392]:
#Make sure all rows left has image url, this should read True
archive_clean['expanded_urls'].str.match(r'.*,?https://twitter.com/dog_rates/status/\d{18}/photo/1,?.*', case=False, na=False).all()

True

### Quality Issue #3: check if any tweets are after 1-Aug-2017

#### Define
Convert timestamp column to date-time object, then sort values to see if any time stamp is after 1-Aug-2017

#### Code

In [393]:
archive_clean['timestamp'] = pd.to_datetime(archive_clean['timestamp'])

#### Test
It looks like no time stamps are after 1-Aug-2017

In [394]:
#Check if any timestand equal or after 2-Aug-2017, this should read False
(archive_clean['timestamp'] >= '2-Aug-2017').any()

False

### Quality Issue #4: some `rating_numerator` and `rating_denominator` are parsed incorrectly

#### Part 1
Some of the values in rating_denominator are not 10

In [395]:
odd_denominotor = archive_clean.query('rating_denominator != 10')

In [396]:
odd_denominotor.to_csv('odd_denominotor.csv')

#### Out of those records, some of the rating numenators and denominators are parsed incorrectly from the text, others are just result of multiple dogs. Since there are not many odd records, I just manually check the excel file and find out the tweet_id that need to be dropped.

#### Define
Drop rows with index in the list

#### Code

In [397]:
index_to_drop = [516, 1068, 1165, 1202, 1662, 2335]

In [398]:
archive_clean = archive_clean.drop(index_to_drop)

#### Test
Make sure all the rows with the index in `index_to_drop` are dropped

In [399]:
#This shuold read False
archive_clean.index.isin(index_to_drop).any()

False

#### Part2 
Some rating result are out of range

Insert a new column `rating_result` as a result of `rating_numerator` devided by `rating_denominator`

In [400]:
archive_clean['rating_result'] = archive_clean['rating_numerator'] / archive_clean['rating_denominator']

Take a look at all the `rating_result`

In [401]:
archive_clean['rating_result'].value_counts().sort_index()

0.0        2
0.1        4
0.2        9
0.3       19
0.4       14
0.5       33
0.6       32
0.7       51
0.8       93
0.9      149
1.0      414
1.1      392
1.2      433
1.3      235
1.4       29
2.6        1
2.7        1
7.5        1
42.0       1
177.6      1
Name: rating_result, dtype: int64

In [402]:
archive_clean.query('rating_result <= 0.1')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,rating_result
315,835152434251116546,NaN,NaN,2017-02-24 15:40:31,"<a href=""http://twitter.com/download/iphone"" r...",When you're so blinded by your systematic plag...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/835152434...,0,10,None,None,None,None,None,0.0
1016,746906459439529985,7.468859e+17,4.196984e+09,2016-06-26 03:22:31,"<a href=""http://twitter.com/download/iphone"" r...","PUPDATE: can't see any. Even if I could, I cou...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/746906459...,0,10,None,None,None,None,None,0.0
1869,675153376133427200,NaN,NaN,2015-12-11 03:21:23,"<a href=""http://twitter.com/download/iphone"" r...",What kind of person sends in a picture without...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/675153376...,1,10,None,None,None,None,None,0.1
2091,670783437142401025,NaN,NaN,2015-11-29 01:56:48,"<a href=""http://twitter.com/download/iphone"" r...",Flamboyant pup here. Probably poisonous. Won't...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/670783437...,1,10,None,None,None,None,None,0.1
2261,667549055577362432,NaN,NaN,2015-11-20 03:44:31,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Never seen dog like this. Breathes heavy. Tilt...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/667549055...,1,10,None,None,None,None,None,0.1
2338,666104133288665088,NaN,NaN,2015-11-16 04:02:55,"<a href=""http://twitter.com/download/iphone"" r...",Not familiar with this breed. No tail (weird)....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666104133...,1,10,None,None,None,None,None,0.1


Those small ratings are parsed properly.

Now take a look at the high `rating_result`

In [403]:
archive_clean.query('rating_result >= 2.6')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,rating_result
695,786709082849828864,NaN,NaN,2016-10-13 23:23:56,"<a href=""http://twitter.com/download/iphone"" r...","This is Logan, the Chow who lived. He solemnly...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/786709082...,75,10,Logan,None,None,None,None,7.5
763,778027034220126208,NaN,NaN,2016-09-20 00:24:34,"<a href=""http://twitter.com/download/iphone"" r...",This is Sophie. She's a Jubilant Bush Pupper. ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/778027034...,27,10,Sophie,None,None,pupper,None,2.7
979,749981277374128128,NaN,NaN,2016-07-04 15:00:45,"<a href=""https://about.twitter.com/products/tw...",This is Atticus. He's quite simply America af....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/749981277...,1776,10,Atticus,None,None,None,None,177.6
1712,680494726643068929,NaN,NaN,2015-12-25 21:06:00,"<a href=""http://twitter.com/download/iphone"" r...",Here we have uncovered an entire battalion of ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/680494726...,26,10,None,None,None,None,None,2.6
2074,670842764863651840,NaN,NaN,2015-11-29 05:52:33,"<a href=""http://twitter.com/download/iphone"" r...",After so many requests... here you go.\n\nGood...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/670842764...,420,10,None,None,None,None,None,42.0


It apprears they are eith parsed incorrectly, so the auther just gives a absurd number, it is safe to drop those four rows.

#### Code

In [404]:
archive_clean = archive_clean[archive_clean['rating_result'] < 2.6]

#### Test 
Make sure all the `rating_result` less than 2.6

In [405]:
#This should read True
(archive_clean['rating_result'] < 2.6).all()

True

### Quality issue #5: `tweet_id` should be changed to data type `str`

In [426]:
archive_clean['tweet_id'] = archive_clean['tweet_id'].astype('str')

In [427]:
archive_clean.dtypes

tweet_id                         object
in_reply_to_status_id           float64
timestamp                datetime64[ns]
rating_result                   float64
stage                            object
dtype: object

### Tidy issue #1: the columns `'doggo', 'floofer', 'pupper', 'puppo'` are values of variable `stage`

#### Define
Convert `None` to 0, `doggo', 'floofer', 'pupper', 'puppo'` to 1 in their own columns, drop rows with multiple nonezero values

#### Code

In [406]:
cols = ['doggo', 'floofer', 'pupper', 'puppo']

In [407]:
#Replace 'None' with 0, other values with 1
archive_clean[cols] = archive_clean[cols].replace({'None':0, 'doggo':1, 'pupper':1, 'puppo':1, 'floofer':1})

In [408]:
archive_clean[cols].sum(axis=1).value_counts()

0    1623
1     276
2      10
dtype: int64

There are 10 rows with two stages, which is a very small sample of this data set. As a result, I am going classify them as outlier and drop them.

In [409]:
archive_clean = archive_clean[archive_clean[cols].sum(axis=1) < 2]

In [410]:
#Make sure the outliers have been dropped, this should return nothing
archive_clean[(archive_clean[cols].sum(axis=1)) > 1]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,rating_result


Create a new column stage, the values should be NaN if none of the four columns is 1, or the column name if it is 1. 

In [411]:
archive_clean['stage'] = archive_clean[cols].apply(lambda x: np.NaN if x.sum()==0 else x.idxmax(), axis=1)

#### Test
Make sure the 1s in `'doggo', 'floofer', 'pupper', 'puppo'` columns match their columns in the `stage` column

In [412]:
#THis should read True
((sum(archive_clean['pupper']) == archive_clean['stage'].value_counts()['pupper']) 
& (sum(archive_clean['doggo']) == archive_clean['stage'].value_counts()['doggo']) 
& (sum(archive_clean['puppo']) == archive_clean['stage'].value_counts()['puppo'])
& (sum(archive_clean['floofer'])== archive_clean['stage'].value_counts()['floofer']))

True

### Tidy issue #2: remove unnesscery columns

In [413]:
cols_keep = ['tweet_id','in_reply_to_status_id','timestamp','rating_result','stage']

In [414]:
archive_clean = archive_clean[cols_keep]

## tweet_json table

In [415]:
#Adjust jupyter notebook to normal display 
pd.set_option('display.max_colwidth', 50)

In [416]:
tweet_json_clean = tweet_json.copy()

In [417]:
tweet_json_clean.dtypes

contributors                      object
coordinates                       object
created_at                        object
display_text_range                object
entities                          object
extended_entities                 object
favorite_count                   float64
favorited                        float64
full_text                         object
geo                               object
id                               float64
id_str                            object
in_reply_to_screen_name           object
in_reply_to_status_id             object
in_reply_to_status_id_str         object
in_reply_to_user_id               object
in_reply_to_user_id_str           object
is_quote_status                  float64
lang                              object
place                             object
possibly_sensitive               float64
possibly_sensitive_appealable    float64
retweet_count                    float64
retweeted                        float64
source          

### Quality Issue #1: `favorite_count` , `retweet_count` should be converted to integer

#### Code

In [418]:
tweet_json_clean['favorite_count'] = tweet_json_clean['favorite_count'].astype('int64')

In [419]:
tweet_json_clean['retweet_count'] = tweet_json_clean['retweet_count'].astype('int64')

#### Test

In [421]:
tweet_json_clean.dtypes

contributors                      object
coordinates                       object
created_at                        object
display_text_range                object
entities                          object
extended_entities                 object
favorite_count                     int64
favorited                        float64
full_text                         object
geo                               object
id                                 int64
id_str                            object
in_reply_to_screen_name           object
in_reply_to_status_id             object
in_reply_to_status_id_str         object
in_reply_to_user_id               object
in_reply_to_user_id_str           object
is_quote_status                  float64
lang                              object
place                             object
possibly_sensitive               float64
possibly_sensitive_appealable    float64
retweet_count                      int64
retweeted                        float64
source          

### Tidy issue #1: unnessery table
This table describe the attributes of each tweet_id, which is the same as `archive_clean` table, I will merge `favorite_count` and `retweet_count` columns to `archive_clean` table

#### Define
Use merge function

#### Code

In [422]:
tweet_json_clean.shape

(2339, 32)

In [423]:
archive_clean.shape

(1899, 5)

In [424]:
cols_merge = ['id_str', 'favorite_count', 'retweet_count']

In [441]:
tweet_merge = archive_clean.merge(tweet_json_clean[cols_merge], how='left', left_on='tweet_id', right_on='id_str')

In [442]:
sum(tweet_merge['id_str'].isnull())

3

Only 3 rows in `archive_clean` can't be matched with `tweet_json_clean` table, I will drop them

In [443]:
tweet_merge = tweet_merge[tweet_merge['id_str'].notnull()]

In [444]:
#Drop 'id_str' because it duplicates 'tweet_id'
tweet_merge = tweet_merge.drop('id_str', axis=1)

#### Test
Make sure `favorite_count` and `retweet_count` columns are without null values

In [445]:
#This should read 0
sum(tweet_merge['favorite_count'].isnull()) + sum(tweet_merge['retweet_count'].isnull())

0

In [446]:
tweet_merge.dtypes

tweet_id                         object
in_reply_to_status_id           float64
timestamp                datetime64[ns]
rating_result                   float64
stage                            object
favorite_count                  float64
retweet_count                   float64
dtype: object

In [447]:
tweet_merge.head()

,tweet_id,in_reply_to_status_id,timestamp,rating_result,stage,favorite_count,retweet_count
0,892420643555336193,NaN,2017-08-01 16:23:56,1.3,NaN,37713.0,8217.0
1,892177421306343426,NaN,2017-08-01 00:17:27,1.3,NaN,32385.0,6076.0
2,891815181378084864,NaN,2017-07-31 00:18:03,1.2,NaN,24382.0,4019.0
3,891689557279858688,NaN,2017-07-30 15:58:51,1.3,NaN,41022.0,8363.0
4,891327558926688256,NaN,2017-07-29 16:00:24,1.2,NaN,39214.0,9076.0
